# Modelling NIQS Hardware

In [1]:
import numpy as np
import qiskit as qk
from qiskit.quantum_info import DensityMatrix
from scipy.linalg import sqrtm

np.set_printoptions(precision=2)

In [13]:
def partial_trace(X, discard_first = True):
    d = X.shape[0]
    d_red = int(np.sqrt(d))
    Y = np.zeros((d_red, d_red), dtype = "complex128")
    I = np.eye(d_red)
    
    for i in range(d_red):
        basis_vec = np.zeros((d_red, 1),  dtype = "complex128")
        basis_vec[i, 0] = 1
        
        if discard_first:
            basis_vec = np.kron(basis_vec, I)
        else:
            basis_vec = np.kron(I, basis_vec)
        
        Y = Y + basis_vec.T@X@basis_vec
    
    return Y


def state_fidelity(A, B):
    sqrtA = sqrtm(A)
    fidelity = np.trace(sqrtm(sqrtA@B@sqrtA))
    return fidelity


def apply_map(state, choi):
    d = state.shape[0]
    
    #reshuffle
    choi = choi.reshape(d,d,d,d).swapaxes(1,2).reshape(d**2, d**2)
    
    #flatten
    state = state.reshape(-1, 1)
    
    state = (choi@state).reshape(d, d)
    return state
    

def prepare_input(config):
    """1 = |0>, 2 = |1>, 3 = |+>, 4 = |->, 5 = |+i>, 6 = |-i>"""
    n = len(config)
    circuit = qk.QuantumCircuit(n)
    for i, gate in enumerate(config):
        if gate == 2:
            circuit.x(i)
        if gate == 3:
            circuit.h(i)
        if gate == 4:
            circuit.x(i)
            circuit.h(i)
        if gate == 5:
            circuit.h(i)
            circuit.s(i)
        if gate == 6:
            circuit.x(i)
            circuit.h(i)
            circuit.s(i)
        
            
    rho = DensityMatrix(circuit)
    return rho.data


def generate_choi(X):
    d = int(np.sqrt(X.shape[0]))  # dim of Hilbert space
    I = np.eye(d)

    #partial trace
    Y = partial_trace(X@(X.conj().T), discard_first = True)
    sqrtYinv = np.linalg.inv(sqrtm(Y))

    #choi
    choi = np.kron(I, sqrtYinv)@X@(X.conj().T)@np.kron(I, sqrtYinv)
    
    return choi


class Adam():
    def __init__(self, lr=0.01, beta1=0.9, beta2=0.999, eps=1e-8):
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.m = None
        self.v = None
        self.t = None

    def initialize(self, dims):
        self.m = []
        self.v = []
        self.t = 0

        for dim in dims:
            self.m.append(np.zeros(dim, dtype="complex128"))
            self.v.append(np.zeros(dim, dtype="complex128"))

    def __call__(self, weight_gradient_list):
        self.t += 1
        weight_gradient_modified = []

        for grad, m_, v_ in zip(weight_gradient_list, self.m, self.v):
            m_[:] = self.beta1 * m_ + (1 - self.beta1) * grad
            v_[:] = self.beta2 * v_ + (1 - self.beta2) * grad**2

            m_hat = m_ / (1 - self.beta1**self.t)
            v_hat = v_ / (1 - self.beta2**self.t)
            grad_modified = m_hat / (np.sqrt(v_hat) + self.eps)
            weight_gradient_modified.append(grad_modified)

        return weight_gradient_modified

In [22]:
h = 1e-4
lr = 0.1
optim = Adam()


#1 = |0>, 2 = |1>, 3 = |+>, 4 = |->, 5 = |i+>, 6 = |i->
state_input = prepare_input([3, 4])

n = 2
d = 2**n
optim.initialize(dims = [(d**2, d**2)])

np.random.seed(42)

X_target = np.random.normal(0, 1, (d**2, 1)) + 1j*np.random.normal(0, 1, (d**2, 1))
choi_target = generate_choi(X_target)
state_target = apply_map(state_input, choi_target)

X_zero = np.random.normal(0, 1, (d**2, d**2)) + 1j*np.random.normal(0, 1, (d**2, d**2))
choi_zero = generate_choi(X_zero)
state_zero = apply_map(state_input, choi_zero)
fid_zero = state_fidelity(state_zero, state_target)

for steps in range(100):  
    grad_matrix = np.zeros((d**2, d**2), dtype="complex128")

    for i in range(d**2):
        for j in range(d**2):
            X_plus = np.copy(X_zero)
            X_plus[i, j] += h
            choi_plus = generate_choi(X_plus)
            state_plus = apply_map(state_input, choi_plus)
            fid_plus = state_fidelity(state_plus, state_target)
            
            X_minus = np.copy(X_zero)
            X_minus[i, j] -=h
            choi_minus = generate_choi(X_minus)
            state_minus = apply_map(state_input, choi_minus)
            fid_minus = state_fidelity(state_minus, state_target)
            
            grad = (fid_plus-fid_minus)/(2*h)
            grad_matrix[i, j] += grad 
            
            X_plus = np.copy(X_zero)
            X_plus[i, j] += 1j*h
            choi_plus = generate_choi(X_plus)
            state_plus = apply_map(state_input, choi_plus)
            fid_plus = state_fidelity(state_plus, state_target)
            
            X_minus = np.copy(X_zero)
            X_minus[i, j] -= 1j*h
            choi_minus = generate_choi(X_minus)
            state_minus = apply_map(state_input, choi_minus)
            fid_minus = state_fidelity(state_minus, state_target)
            
            grad = 1j*(fid_plus-fid_minus)/(2*h)
            grad_matrix[i, j] += grad
            
    
            
    grad_matrix = optim([grad_matrix])[0]
    X_zero += lr*grad_matrix
    
    choi_zero = generate_choi(X_zero)
    state_zero = apply_map(state_input, choi_zero)
    fid_zero = state_fidelity(state_zero, state_target)
    
    print(f"{steps}: {np.abs(fid_zero):.4f}")

0: 0.5485
1: 0.6090
2: 0.6612
3: 0.7057
4: 0.7423
5: 0.7712
6: 0.7947
7: 0.8123
8: 0.8263
9: 0.8369
10: 0.8457
11: 0.8523
12: 0.8592
13: 0.8644
14: 0.8686
15: 0.8729
16: 0.8769
17: 0.8807
18: 0.8848
19: 0.8890
20: 0.8926
21: 0.8965
22: 0.9000
23: 0.9034
24: 0.9066
25: 0.9096
26: 0.9126
27: 0.9154
28: 0.9179
29: 0.9203
30: 0.9224
31: 0.9245
32: 0.9264
33: 0.9281
34: 0.9296
35: 0.9311
36: 0.9325
37: 0.9338
38: 0.9351
39: 0.9363
40: 0.9375
41: 0.9387
42: 0.9398
43: 0.9410
44: 0.9420
45: 0.9430
46: 0.9442
47: 0.9454
48: 0.9466
49: 0.9479
50: 0.9490
51: 0.9501
52: 0.9511
53: 0.9520
54: 0.9528
55: 0.9536
56: 0.9543
57: 0.9549
58: 0.9555
59: 0.9561
60: 0.9566
61: 0.9571
62: 0.9576
63: 0.9581
64: 0.9586
65: 0.9591
66: 0.9596
67: 0.9600
68: 0.9605
69: 0.9609
70: 0.9613
71: 0.9617
72: 0.9620
73: 0.9623
74: 0.9626
75: 0.9631
76: 0.9635
77: 0.9640
78: 0.9645
79: 0.9649
80: 0.9653
81: 0.9657
82: 0.9661
83: 0.9665
84: 0.9669
85: 0.9673
86: 0.9678
87: 0.9683
88: 0.9688
89: 0.9694
90: 0.9698
91: 0.970